<a href="https://colab.research.google.com/github/tina60239/MarketWatchCat/blob/main/20250315_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
url = "https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX"
payload = {
    "date": "20250314",
    "type": "ALLBUT0999",
    "response": "json"
}
res = requests.get(url, params = payload)
res

<Response [200]>

In [ ]:
import json
jd = json.loads(res.text) #json.loads()將json字串轉成 python dict
df = pd.DataFrame(jd['tables'][8]['data'], columns = jd['tables'][8]['fields'])
df = df[['證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價']] #挑選需要的欄位
df = df.apply(lambda x : x.str.replace(",", "")) #將逗號刪除
df.insert(0, "日期", "20250314") #增加日期欄位
df

,日期,證券代號,成交股數,開盤價,最高價,最低價,收盤價
0,20250314,0050,14434475,181.10,181.15,179.55,180.50
1,20250314,0051,41374,76.25,76.50,76.10,76.30
2,20250314,0052,463826,178.50,179.20,177.10,178.10
3,20250314,0053,10340,96.55,96.55,95.55,96.35
4,20250314,0055,56090,28.48,28.62,28.48,28.49
...,...,...,...,...,...,...,...
1281,20250314,9944,80194,20.00,20.10,19.95,20.10
1282,20250314,9945,21004707,38.75,38.80,37.90,38.00
1283,20250314,9946,651464,24.80,24.95,24.70,24.90
1284,20250314,9955,1475199,31.05,31.30,30.30,30.65


In [ ]:
jd['stat']

'OK'

In [ ]:
import sqlite3 as lite
sql = '''
CREATE TABLE IF NOT EXISTS 每日收盤行情 (
    日期     TEXT,
    證券代號 TEXT,
    成交股數	TEXT,
    開盤價	TEXT,
    最高價	TEXT,
    最低價	TEXT,
    收盤價   TEXT,
    PRIMARY KEY(日期, 證券代號)
);
'''

con = lite.connect("台股資料.db") #開啟資料庫連結
cur = con.cursor() #取得資料庫執行游標
cur.execute(sql) #執行sql語法
con.commit()#確認寫入, 當資料庫有新增、修改、刪除資料庫任何一個動作，皆要執行確認寫入
cur.close() #關閉游標
con.close() #關閉資料庫連結

### 新增資料到資料庫

In [ ]:
import sqlite3 as lite
sql = '''
INSERT INTO 每日收盤行情 (日期, 證券代號, 成交股數, 開盤價, 最高價, 最低價, 收盤價)
VALUES (?, ?, ?, ?, ?, ?, ?);
'''

con = lite.connect("台股資料.db") #開啟資料庫連結
cur = con.cursor() #取得資料庫執行游標
cur.executemany(sql, df.values.tolist()) #執行sql語法
con.commit()#確認寫入, 當資料庫有新增、修改、刪除資料庫任何一個動作，皆要執行確認寫入
cur.close() #關閉游標
con.close() #關閉資料庫連結

In [ ]:
import sqlite3 as lite
sql = '''
SELECT * FROM 每日收盤行情;
'''

con = lite.connect("台股資料.db") #開啟資料庫連結
cur = con.cursor() #取得資料庫執行游標
cur.execute(sql) #執行sql語法
rows = cur.fetchall() #取得全部資料
header = [item[0] for item in cur.description] #取得資料表標題
cur.close() #關閉游標
con.close() #關閉資料庫連結
df1 = pd.DataFrame(rows, columns = header)
df1

,日期,證券代號,成交股數,開盤價,最高價,最低價,收盤價
0,20250314,0050,14434475,181.10,181.15,179.55,180.50
1,20250314,0051,41374,76.25,76.50,76.10,76.30
2,20250314,0052,463826,178.50,179.20,177.10,178.10
3,20250314,0053,10340,96.55,96.55,95.55,96.35
4,20250314,0055,56090,28.48,28.62,28.48,28.49
...,...,...,...,...,...,...,...
1281,20250314,9944,80194,20.00,20.10,19.95,20.10
1282,20250314,9945,21004707,38.75,38.80,37.90,38.00
1283,20250314,9946,651464,24.80,24.95,24.70,24.90
1284,20250314,9955,1475199,31.05,31.30,30.30,30.65


In [ ]:
%pwd

'C:\\Users\\USER\\Desktop\\157102_python金融資料庫'

In [ ]:
import datetime as dt
dt.datetime.today(), dt.datetime.now()

(datetime.datetime(2025, 3, 15, 11, 16, 34, 339950),
 datetime.datetime(2025, 3, 15, 11, 16, 34, 339949))

In [ ]:
#日期時間計算
dt.datetime.today() - dt.timedelta(days = 5)

datetime.datetime(2025, 3, 10, 11, 17, 39, 85325)

In [ ]:
base = dt.datetime.today() - dt.timedelta(days = 5) #取得指定日期基準日
while base < dt.datetime.today(): #時間比較
    print(base.strftime("%Y%m%d"))
    base += dt.timedelta(days = 1) #日期累加

20250310
20250311
20250312
20250313
20250314


In [ ]:
t = 10
while t > -1:
    min, sec = divmod(t, 60)
    if t > 0:
        print(f"{min:02d}:{sec:02d}", end = '\r')
    if t == 0:
        print(f"{min:02d}:{sec:02d}", end = '\n')
    t -= 1
    time.sleep(1)

00:00


In [ ]:
sql = '''
INSERT INTO {} {}
VALUES ({});
'''
sql.format('每日收盤行情', tuple(df.columns.tolist()), ','.join('?'*len(df.columns)))

"\nINSERT INTO 每日收盤行情 ('日期', '證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價') \nVALUES (?,?,?,?,?,?,?);\n"

In [ ]:
df.columns

Index(['日期', '證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價'], dtype='object')

In [ ]:
[f'{item} TEXT' for item in df.columns]

['日期 TEXT',
 '證券代號 TEXT',
 '成交股數 TEXT',
 '開盤價 TEXT',
 '最高價 TEXT',
 '最低價 TEXT',
 '收盤價 TEXT']

In [ ]:
sql = '''
CREATE TABLE IF NOT EXISTS {} (
    {},
    PRIMARY KEY({})
);
'''
sql.format('每日收盤行情',
           ','.join(['{} TEXT'.format(item) for item in df.columns]),
           ','.join(['日期', '證券代號']))

'\nCREATE TABLE IF NOT EXISTS 每日收盤行情 (\n    日期 TEXT,證券代號 TEXT,成交股數 TEXT,開盤價 TEXT,最高價 TEXT,最低價 TEXT,收盤價 TEXT,\n    PRIMARY KEY(日期,證券代號)\n);\n'

In [ ]:
import requests
import pandas as pd
import json
import time
import sqlite3 as lite

class operateDB():
    def __init__(self, dbname, tablename, dbcolumns = None, primarykey = None):
        self.dbname = dbname
        self.tablename = tablename
        self.dbcolumns = dbcolumns
        self.primarykey = primarykey

    def rundb(self, sql, data = None, readdata = False):
        print(sql)
        con = lite.connect(self.dbname) #開啟資料庫連結
        cur = con.cursor() #取得資料庫執行游標

        if data == None:
            cur.execute(sql)
        else:
            cur.executemany(sql, data) #執行sql語法
        if not readdata:
            con.commit()#確認寫入, 當資料庫有新增、修改、刪除資料庫任何一個動作，皆要執行確認寫入
        else:
            rows = cur.fetchall()
            header = [item[0] for item in cur.description]
            df = pd.DataFrame(rows, columns = header)

        cur.close() #關閉游標
        con.close() #關閉資料庫連結

        if readdata:
            return df
        else:
            return pd.DataFrame()

    def createtable(self):
        sql = '''
        CREATE TABLE IF NOT EXISTS {} (
            {},
            PRIMARY KEY({})
        );
        '''
        sql = sql.format(self.tablename,
                           ','.join(['{} TEXT'.format(item) for item in self.dbcolumns]),
                           ','.join(self.primarykey))
        #print(sql)
        self.rundb(sql)

    def insertdata(self, df):
        self.createtable()
        sql = '''
        INSERT OR REPLACE INTO {} {}
        VALUES ({});
        '''
        sql = sql.format(self.tablename,
                         tuple(self.dbcolumns),
                         ','.join('?'*len(self.dbcolumns)))
        self.rundb(sql, df.values.tolist())


def countdown(t):
    while t > -1:
        min, sec = divmod(t, 60)
        if t > 0:
            print(f"{min:02d}:{sec:02d}", end = '\r')
        if t == 0:
            print(f"{min:02d}:{sec:02d}", end = '\n')
        t -= 1
        time.sleep(1)

def quote(date):
    url = "https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX"
    payload = {
        "date": date,
        "type": "ALLBUT0999",
        "response": "json"
    }
    res = requests.get(url, params = payload)
    jd = json.loads(res.text) #json.loads()將json字串轉成 python dict
    if jd['stat'].lower() == "ok": #確認交易日有資料
        df = pd.DataFrame(jd['tables'][8]['data'], columns = jd['tables'][8]['fields'])
        df = df[['證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價']] #挑選需要的欄位
        df = df.apply(lambda x : x.str.replace(",", "")) #將逗號刪除
        df.insert(0, "日期", date) #增加日期欄位
        return df #有資料時回傳
    else:
        return pd.DataFrame() #沒資料時回傳空的表格

data = []
base = dt.datetime.today() - dt.timedelta(days = 5) #取得指定日期基準日
while base < dt.datetime.today(): #時間比較
    date = base.strftime("%Y%m%d")
    print(date)
    df0 = quote(date)
    if not df0.empty: #留下有資料
        data.append(df0) #帶入日期取得當天交易日的行情資料
    countdown(10) #每次抓取資料後，暫停10秒
    base += dt.timedelta(days = 1) #日期累加
df = pd.concat(data, ignore_index = True)
df

NameError: name 'dt' is not defined

In [ ]:
df.columns.tolist()

['日期', '證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價']

In [ ]:
import sqlite3 as lite
class operateDB():
    def __init__(self, dbname, tablename, dbcolumns = None, primarykey = None):
        self.dbname = dbname
        self.tablename = tablename
        self.dbcolumns = dbcolumns
        self.primarykey = primarykey

    def rundb(self, sql, data = None, readdata = False):
        print(sql)
        con = lite.connect(self.dbname) #開啟資料庫連結
        cur = con.cursor() #取得資料庫執行游標

        if data == None:
            cur.execute(sql)
        else:
            cur.executemany(sql, data) #執行sql語法
        if not readdata:
            con.commit()#確認寫入, 當資料庫有新增、修改、刪除資料庫任何一個動作，皆要執行確認寫入
        else:
            rows = cur.fetchall()
            header = [item[0] for item in cur.description]
            df = pd.DataFrame(rows, columns = header)

        cur.close() #關閉游標
        con.close() #關閉資料庫連結

        if readdata:
            return df
        else:
            return pd.DataFrame()

    def createtable(self):
        sql = '''
        CREATE TABLE IF NOT EXISTS {} (
            {},
            PRIMARY KEY({})
        );
        '''
        sql = sql.format(self.tablename,
                           ','.join(['{} TEXT'.format(item) for item in self.dbcolumns]),
                           ','.join(self.primarykey))
        #print(sql)
        self.rundb(sql)

    def insertdata(self, df):
        self.createtable()
        sql = '''
        INSERT OR REPLACE INTO {} {}
        VALUES ({});
        '''
        sql = sql.format(self.tablename,
                         tuple(self.dbcolumns),
                         ','.join('?'*len(self.dbcolumns)))
        self.rundb(sql, df.values.tolist())



db = operateDB("台股資料.db", "每日收盤行情", df.columns.tolist(), ['日期', '證券代號'])
db.insertdata(df)


        CREATE TABLE IF NOT EXISTS 每日收盤行情 (
            日期 TEXT,證券代號 TEXT,成交股數 TEXT,開盤價 TEXT,最高價 TEXT,最低價 TEXT,收盤價 TEXT,
            PRIMARY KEY(日期,證券代號)
        );
        

        INSERT OR REPLACE INTO 每日收盤行情 ('日期', '證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價') 
        VALUES (?,?,?,?,?,?,?);
        


OperationalError: database is locked

In [ ]:
df.columns

Index(['日期', '證券代號', '成交股數', '開盤價', '最高價', '最低價', '收盤價'], dtype='object')